In [1]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb
import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import numpy as np

In [2]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

TensorBoard version:  2.7.0


In [3]:
env_list = ['atari_seaquest', 'atari_enduro', 'atari_pong']
agent_list = ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'patch_size_1', 'patch_size_4', 'patch_size_16']
seed_list = ['1', '2', '3']
metric_list = ['scalars/train_return', 'scalars/eval_resample_jitter_inter_avg_return', 'scalars/eval_resample_jitter_extra_avg_return', 'scalars/eval_rgb_avg_return']
dir_path = '../split_dreamer_logs/logs/'
def extract_score():
    score_array = np.zeros((len(env_list), len(agent_list), len(metric_list), len(seed_list)))
    for env_idx, env in enumerate(env_list):
        for agent_idx, agent in enumerate(agent_list):
            for metric_idx, metric in enumerate(metric_list):
                for seed_idx, seed in enumerate(seed_list):

                    file_path = os.path.join(dir_path, env, agent, seed)
                    print(file_path)
                    # read from summary writer
                    event_acc = EventAccumulator(file_path, size_guidance={'tensors': 0})
                    event_acc.Reload()
                    w, s, t = event_acc.Tensors(metric)[-1]
                    score_array[env_idx, agent_idx, metric_idx, seed_idx] = tf.make_ndarray(t)
    return score_array
#                 dataframe = pd.DataFrame([(w, s, tf.make_ndarray(t)) for w, s, t in event_acc.Tensors('scalars/eval_resample_jitter_extra_avg_return')],
#                 columns=['wall_time', 'step', 'tensor'])
# Save
score_array = extract_score()
np.save(os.path.join(dir_path, "scores.npy"), score_array)
# Load
score_array = np.load(os.path.join(dir_path, "scores.npy"))

../split_dreamer_logs/logs/atari_seaquest/agent_1/1
../split_dreamer_logs/logs/atari_seaquest/agent_1/2
../split_dreamer_logs/logs/atari_seaquest/agent_1/3
../split_dreamer_logs/logs/atari_seaquest/agent_1/1
../split_dreamer_logs/logs/atari_seaquest/agent_1/2
../split_dreamer_logs/logs/atari_seaquest/agent_1/3
../split_dreamer_logs/logs/atari_seaquest/agent_1/1
../split_dreamer_logs/logs/atari_seaquest/agent_1/2
../split_dreamer_logs/logs/atari_seaquest/agent_1/3
../split_dreamer_logs/logs/atari_seaquest/agent_1/1
../split_dreamer_logs/logs/atari_seaquest/agent_1/2
../split_dreamer_logs/logs/atari_seaquest/agent_1/3
../split_dreamer_logs/logs/atari_seaquest/agent_2/1
../split_dreamer_logs/logs/atari_seaquest/agent_2/2
../split_dreamer_logs/logs/atari_seaquest/agent_2/3
../split_dreamer_logs/logs/atari_seaquest/agent_2/1
../split_dreamer_logs/logs/atari_seaquest/agent_2/2
../split_dreamer_logs/logs/atari_seaquest/agent_2/3
../split_dreamer_logs/logs/atari_seaquest/agent_2/1
../split_dre

In [4]:
score_mean = np.int_(list(np.mean(score_array, axis = 3)))
score_std = np.int_(list(np.std(score_array, axis = 3)))

In [5]:
print(score_mean[1,:,:].shape)

(7, 4)


In [6]:
columns_name = ['inter_avg_return', 'extra_avg_return', 'rgb_avg_return']
rows_name = ['Baseline', 'Upper Bound (Trained with full range jittering)', 'SPLIT Patch Size: 8' , 'SPLIT Patch Size: 8 & Fixed Beta' , 'SPLIT Patch Size: 1' , 'SPLIT Patch Size: 4', 'SPLIT Patch Size: 16']
seaquest_returns = pd.DataFrame(score_mean[0, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
enduro_returns = pd.DataFrame(score_mean[1, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
pong_returns = pd.DataFrame(score_mean[2, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)

seaquest_returns_std = pd.DataFrame(score_std[0, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
enduro_returns_std = pd.DataFrame(score_std[1, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
pong_returns_std = pd.DataFrame(score_std[2, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)



In [7]:
seaquest_returns.style.highlight_max(color='red',axis=0)
# Seaquest Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,4160,110,3905
Upper Bound (Trained with full range jittering),2576,2468,2746
SPLIT Patch Size: 8,8184,293,8878
SPLIT Patch Size: 8 & Fixed Beta,4655,175,4720
SPLIT Patch Size: 1,13965,1525,13899
SPLIT Patch Size: 4,6598,539,6863
SPLIT Patch Size: 16,8598,988,8226


In [8]:
seaquest_returns_std

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,1412,30,1463
Upper Bound (Trained with full range jittering),725,673,756
SPLIT Patch Size: 8,1814,62,2059
SPLIT Patch Size: 8 & Fixed Beta,2179,122,2189
SPLIT Patch Size: 1,12490,535,11696
SPLIT Patch Size: 4,899,119,1030
SPLIT Patch Size: 16,1394,400,642


In [9]:
enduro_returns.style.highlight_max(color='red',axis=0)
# Enduro Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,1490,180,1417
Upper Bound (Trained with full range jittering),903,730,975
SPLIT Patch Size: 8,1476,393,1474
SPLIT Patch Size: 8 & Fixed Beta,1574,248,1476
SPLIT Patch Size: 1,1788,632,1956
SPLIT Patch Size: 4,1454,438,1681
SPLIT Patch Size: 16,2018,665,2099


In [ ]:
enduro_returns_std

In [10]:
pong_returns.style.highlight_max(color='red',axis=0)
# Pong Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,19,-14,19
Upper Bound (Trained with full range jittering),17,15,16
SPLIT Patch Size: 8,17,6,17
SPLIT Patch Size: 8 & Fixed Beta,17,-9,17
SPLIT Patch Size: 1,15,1,15
SPLIT Patch Size: 4,10,4,10
SPLIT Patch Size: 16,19,15,19


In [ ]:
pong_returns_std